In [31]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import gmean
from pandas import DataFrame


In [32]:
df_Results = pd.read_pickle("UsableData/Results.pkl")
df_Results.head(5)

,Market Cap,Employees,Book Value,Equal,Random,Variance
1995-12-31,1,1,1,1,1,1
1996-01-31,1.055732,1.037344,1.027627,1.00793,1.007622,0.975108
1996-02-29,1.06463,1.050292,1.042715,1.027357,1.027117,1.001764
1996-03-31,1.036932,1.029997,1.026332,1.018903,1.018787,1.006524
1996-04-30,1.043941,1.036461,1.032508,1.024496,1.024371,1.011145


In [41]:
# Based on 3.1 Finance Introduction
total_cumulative_return = (df_Results.iloc[-1]-df_Results.iloc[1])*100
total_cumulative_return

Market Cap    544.274363
Employees     798.356717
Book Value    660.385263
Equal         879.591946
Random        910.748051
Variance      147.831215
dtype: object

In [42]:
# Based on 3.1 Finance Introduction
mean_yearly_return = (((1+(total_cumulative_return/100)))**(1/10)-1)*100
mean_yearly_return

Market Cap    20.477816
Employees      24.55033
Book Value    22.490771
Equal         25.633229
Random        26.027201
Variance       9.500374
dtype: object

In [34]:
df_Returns_Strat = df_Results.pct_change()*100
df_Returns_Strat.dropna(axis=0, inplace=True)
df_Returns_Strat.head(5)

,Market Cap,Employees,Book Value,Equal,Random,Variance
1996-01-31,5.573161,3.734411,2.762652,0.793015,0.762178,-2.489193
1996-02-29,0.842857,1.248215,1.468301,1.927407,1.934737,2.733667
1996-03-31,-2.601658,-1.932360,-1.571210,-0.822860,-0.810965,0.475171
1996-04-30,0.675964,0.627619,0.601806,0.548916,0.548082,0.459045
1996-05-31,9.309816,9.269007,9.247201,9.202487,9.201782,9.126401


In [43]:
monthly_vol = np.std(df_Returns_Strat, axis=0)
monthly_vol

Market Cap    5.722701
Employees     6.003602
Book Value    6.051144
Equal         4.563035
Random        4.577106
Variance      4.909149
dtype: float64

In [44]:
# Read yearly return of DE 10 Year Bound reported monthly
DE10Y = pd.read_csv("RawData/DE10Y.csv", index_col="Datum")
DE10Y.index = pd.to_datetime(DE10Y.index, dayfirst=True)
DE10Y = DE10Y[(DE10Y.index >= df_Results.index[1]) & (DE10Y.index <= df_Results.index[-1])]
for row in DE10Y.index:
    string = DE10Y.loc[row, "Zuletzt"].strip("%").replace(".", "").replace(",", ".")
    DE10Y.at[row, "Zuletzt"] = float(string)
DE10Y.head(5)

,Zuletzt,Eröffn.,Hoch,Tief,+/- %
Datum,,,,,
2021-12-01,-0.179,"-0,3490","-0,1710","-0,4090","-47,81%"
2021-11-01,-0.343,"-0,1100","-0,0650","-0,3630","264,89%"
2021-10-01,-0.094,"-0,1970","-0,0640","-0,2430","-50,79%"
2021-09-01,-0.191,"-0,3730","-0,1720","-0,4010","-49,74%"
2021-08-01,-0.38,"-0,4590","-0,3760","-0,5240","-17,03%"


In [46]:
# Based on 3.1 Finance Introduction
risk_free_return = np.mean(DE10Y["Zuletzt"])
risk_free_return

2.7818504823151122

In [47]:
# Based on 3.1 Finance Introduction
sharpe_ratio = pd.Series(index=df_Results.columns)
for strategy in mean_yearly_return.index:
    sharpe_ratio[strategy] = (mean_yearly_return[strategy] - risk_free_return) / monthly_vol[strategy]
sharpe_ratio

Market Cap    3.092240
Employees     3.625903
Book Value    3.257057
Equal         5.007934
Random        5.078613
Variance      1.368572
dtype: float64

In [48]:
# maximum monthly drawdown
max_monthly_drawdown = pd.Series(index=df_Results.columns)
for strategy in df_Returns_Strat.columns:
    max_monthly_drawdown[strategy] = min(df_Returns_Strat[strategy])
max_monthly_drawdown

Market Cap   -22.348892
Employees    -24.473774
Book Value   -22.915366
Equal        -15.604835
Random       -16.063150
Variance     -27.235294
dtype: float64

In [49]:
positive_month = pd.Series(index=df_Results.columns)
for strategy in df_Returns_Strat.columns:
    returns = df_Returns_Strat[strategy]
    returns = returns[returns > 0]
    positive_month[strategy] = len(returns)
positive_month

Market Cap    186.0
Employees     189.0
Book Value    184.0
Equal         200.0
Random        201.0
Variance      189.0
dtype: float64

In [50]:
df_Analysis = DataFrame(index=df_Results.columns)
df_Analysis["Total Cumulative Return"] = total_cumulative_return
df_Analysis["Mean Yearly Return"] = mean_yearly_return
df_Analysis["Volatility"] = monthly_vol
df_Analysis["Sharpe Ratio"] = sharpe_ratio
df_Analysis["Max Drawdown"] = max_monthly_drawdown
df_Analysis["Positive Month"] = positive_month
df_Analysis.to_pickle("UsableData/Analysis.pkl")
df_Analysis

,Total Cumulative Return,Mean Yearly Return,Volatility,Sharpe Ratio,Max Drawdown,Positive Month
Market Cap,544.274363,20.477816,5.722701,3.092240,-22.348892,186.0
Employees,798.356717,24.55033,6.003602,3.625903,-24.473774,189.0
Book Value,660.385263,22.490771,6.051144,3.257057,-22.915366,184.0
Equal,879.591946,25.633229,4.563035,5.007934,-15.604835,200.0
Random,910.748051,26.027201,4.577106,5.078613,-16.063150,201.0
Variance,147.831215,9.500374,4.909149,1.368572,-27.235294,189.0
